In [1]:
import json
import random
import time

import requests
import spacy
import nltk
from bs4 import BeautifulSoup
from colorama import Back, Fore, Style
from fuzzywuzzy import fuzz
from icecream import ic
from newspaper import Article, Config, ArticleException
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from ratelimit import limits, sleep_and_retry
from robin_stocks import robinhood as r
from tqdm import tqdm
import pandas as pd
import ray

nltk.download('vader_lexicon')


# Load secrets
with open('secrets.json') as f:
    secrets = json.load(f)

# Robinhood login
username = secrets['username']
password = secrets['password']
login = r.login(username, password)

# Initialize NLP tools
nltk.download('vader_lexicon', quiet=True)
sia = SentimentIntensityAnalyzer()
nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/grahamwaters/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
cryptocurrencies = [
    'BTC',
    'ETH',
    'DOGE',
    'LTC',
    'ETC',
    'BCH',
    'BSV',
    'XRP',
    'ADA',
    'DOT',
    'UNI',
    'LINK',
    'XLM'
]

In [9]:


# Define your lists here...

sia = SentimentIntensityAnalyzer()

get_company_names = lambda text: list(set([ent.text for ent in nlp(text).ents if ent.label_ == 'ORG' or ent.text in cryptocurrencies]))

calculate_sentiment = lambda text: sia.polarity_scores(text)['compound']

levenshtein_similarity_check = lambda new_article, articles_list: any(fuzz.ratio(new_article, article) > 80 for article in articles_list)

get_crypto_names = lambda article_text: [crypto for crypto in cryptocurrencies if crypto in article_text]

calculate_sentiment_of_sentence_containing_crypto = lambda article_text: sum(sia.polarity_scores(sentence)["compound"] for sentence in article_text.split('. ') for crypto in cryptocurrencies if crypto in sentence) / len([sentence for sentence in article_text.split('. ') for crypto in cryptocurrencies if crypto in sentence]) if [sentence for sentence in article_text.split('. ') for crypto in cryptocurrencies if crypto in sentence] else 0

# Define your functions here...
@ray.remote
def check_for_trade_signals(sentiment_scores):
    trades = []
    for company, sentiment_score in sentiment_scores.items():
        if sentiment_score > 0.05:  # This is an example threshold for buying
            trades.append((company, 'BUY'))
        elif sentiment_score < -0.05:  # This is an example threshold for selling
            trades.append((company, 'SELL'))
    return trades

import newspaper
@ray.remote
def get_articles(url):
    # use newspaper3k to get articles
    paper = newspaper.build(url, memoize_articles=False)
    articles = []
    for article in paper.articles:
        try:
            article.download()
            article.parse()
            articles.append(article.text)
        except:
            continue
    return articles

@ray.remote
def get_sentiment_scores(articles):
    sentiment_scores = {}
    for article in articles:
        sentiment_scores[article] = calculate_sentiment_of_sentence_containing_crypto(article)
    return sentiment_scores

def getTicker(company_name):
    """
    The getTicker function takes in a company name and returns the ticker symbol for that company.
        It does this by using the Yahoo Finance API to search for a given company, then returning 
        the first result's ticker symbol.
    
    :param company_name: Search for the company's ticker symbol
    :return: The company's ticker symbol
    :credit: bruhbruhroblox on GitHub
    :doc-author: Trelent
    """
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code


@ray.remote
def execute_trades(trades):
    profile = r.profiles.load_account_profile()
    holdings = r.build_holdings()
    owned_stocks = list(holdings.keys())
    buying_power = float(profile['buying_power'])
    for trade in tqdm(trades):
        company, action = trade
        try:
            company_ticker = getTicker(company)
        except Exception as e:
            continue
        if company_ticker not in owned_stocks and action == 'SELL':
            continue
        position_size = min(buying_power * 0.01, buying_power)
        if action == 'BUY':
            time.sleep(4)
            trader(company_ticker, action='BUY')
            time.sleep(random.randint(1,4))
        elif action == 'SELL':
            if company not in owned_stocks:
                continue
            time.sleep(4)
            r.orders.order_sell_stop_loss(symbol=company,quantity=0.95*position_size,stopPrice = 0.95*position_size, timeInForce='gfd')


# get_articles_from_news_source
@ray.remote
def get_articles_from_news_source(news_source):
    # use newspaper3k to get articles
    paper = newspaper.build(news_source, memoize_articles=False)
    articles = []
    for article in paper.articles:
        try:
            article.download()
            article.parse()
            # # check the articles for any company, crypto, or stock names with NER
            # article_company_names = get_company_names(article.text)
            # # check if any of the company names are in the article
            # if article_company_names:
            #     # check if any of the company names are in the articles list
            #     if not levenshtein_similarity_check(article_company_names[0], articles):
            #         articles.append(article_company_names[0]) # append the first company name to the articles list
            #     # now we need to check sentiment of the article
            #     # get the sentiment of the article
            #     article_sentiment = calculate_sentiment(article.text)
            #     # check if the sentiment is positive
            #     if article_sentiment > 0.05:
            #         # if the sentiment is positive, we want to buy the stock
            #         # check if we have enough buying power to buy the stock
            #         if float(r.profiles.load_account_profile()['buying_power']) > 0.01:
            #             # buy the stock
            #             # get the ticker for the company
            #             company_ticker = getTicker(article_company_names[0])
            #             # buy the stock
            #             r.orders.order_buy_fractional_by_price(company_ticker, 0.01)
            #             # wait 5 seconds
            #             time.sleep(5)
            #     # check if the sentiment is negative
            #     elif article_sentiment < -0.05:
            #         # if the sentiment is negative, we want to sell the stock
            #         # check if we have any of the stock
            #         if article_company_names[0] in r.build_holdings():
            #             # sell the stock
            #             # get the ticker for the company
            #             company_ticker = getTicker(article_company_names[0])
            #             # sell the stock
            #             r.orders.order_sell_fractional_by_price(company_ticker, 0.01)
            #             # wait 5 seconds
            #             time.sleep(5)
            # just append the article text to the articles list
            articles.append(article.text)
        except:
            continue
    return articles

@ray.remote
def check_for_stop_loss_orders():
    while True:
        try:
            cryptos = r.crypto.get_crypto_positions()
            for crypto in cryptos:
                current_price = r.crypto.get_crypto_quote(crypto['currency']['code'], info='mark_price')
                if current_price is None or not isinstance(current_price, (int, float)):
                    print(f"Error getting current price for {crypto['currency']['code']}")
                    continue
                stop_loss_price = float(current_price) * 0.99
                open_orders = r.orders.get_all_open_crypto_orders()
                for order in open_orders:
                    if order['side'] == 'sell' and order['price'] == stop_loss_price:
                        r.orders.cancel_crypto_order(order['id'])
                r.orders.order_sell_crypto_limit(crypto['currency']['code'], crypto['quantity'], stop_loss_price)
            time.sleep(60)
        except Exception as e:  
            print(e)
            time.sleep(60)
            continue
        
@ray.remote
def get_articles_from_news_sources(news_sources):
    articles = []
    for news_source in news_sources:
        articles.extend(get_articles_from_news_source.remote(news_source))
    return articles

In [4]:
# run this cell to start the ray cluster
# ignore reinitialization errors
ray.shutdown()
ray.init()

2023-06-06 12:42:59,184	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.4.0


In [10]:
# Using Ray to parallelize the execution of trades
# Main loop
    # Define a list of news sources
news_sources = [
    'https://www.drudge.com/',
    'https://www.bloomberg.com/',
    'https://www.marketwatch.com/',
    'https://www.fool.com/',
    'https://www.businessinsider.com/',
    'https://www.axios.com/',
    'https://www.politico.com/',
    'https://www.theatlantic.com/',
    'https://www.newyorker.com/',
    'https://www.economist.com/',
    'https://www.usnews.com/',
    'https://www.nationalreview.com/',
    'https://www.motherjones.com/',
    'https://www.salon.com/',
    'https://www.slate.com/',
    'https://www.vox.com/',
    'https://www.thehill.com/',
    'https://www.vice.com/',
    'https://www.buzzfeed.com/',
    'https://www.dailykos.com/',
    
]
crypto_news_sources = [
    'https://cointelegraph.com/tags/robinhood',
    'https://finance.yahoo.com/news/'
    'https://finance.yahoo.com/cryptocurrencies/',
    'https://www.coindesk.com/',
    'https://cryptonews.com/',
    'https://crypto.news/',
    'https://www.cryptoglobe.com/latest/',
    'https://www.cryptocraft.com/',
    'https://www.cryptopolitan.com/',
    'https://www.reuters.com/technology/cryptocurrency',
    'https://www.coinspeaker.com/',
    'https://cryptopotato.com/crypto-news/',
    'https://www.cryptocointrade.com/',
    'https://www.bloomberg.com/crypto'

]

while True:

    # Get news sources
    news_articles = []
    
    
    # Get articles
    for news_source in news_sources:
        # Get articles from news source
        news_articles += get_articles_from_news_source.remote(news_source)
    
    # Get crypto articles
    for crypto_news_source in crypto_news_sources:
        news_articles += get_articles_from_news_source.remote(crypto_news_source)
    
    # Wait for all articles to be retrieved
    news_articles = ray.get(news_articles)

    # Flatten list of lists
    news_articles = [article for sublist in news_articles for article in sublist]

    # Remove duplicates with Levendstein distance
    news_articles = remove_duplicates.remote(news_articles)

    # find the overall sentiment of sentences containing each of the coins, stocks, and companies in the news articles
    articles = ray.get(news_articles)
    articles = [article for article in articles if article is not None]
    articles = [article for article in articles if article['sentences'] is not None]
    articles = [article for article in articles if article['sentences'] != []]
    articles = [article for article in articles if article['sentences'] != ['']]
    articles = [article for article in articles if article['sentences'] != [' ']]
    articles = [article for article in articles if article['sentences'] != ['  ']]

    # Get sentiment scores
    sentiment_scores = get_sentiment_scores.remote(articles)

    # Check for trade signals
    trade_signals = check_for_trade_signals.remote(sentiment_scores)
    
    # Execute trades
    execute_trades.remote(trade_signals)

    # Check for stop loss orders
    check_for_stop_loss_orders.remote()

    # Wait for 5 minutes
    time.sleep(300)



TypeError: 'ray._raylet.ObjectRef' object is not iterable

(raylet) [2023-06-06 12:49:00,042 E 23325 147234] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-06-06_12-42-51_999086_23201 is over 95% full, available space: 8601669632; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-06-06 12:49:10,045 E 23325 147234] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-06-06_12-42-51_999086_23201 is over 95% full, available space: 8609861632; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-06-06 12:49:20,143 E 23325 147234] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-06-06_12-42-51_999086_23201 is over 95% full, available space: 8579239936; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-06-06 12:49:30,143 E 23325 147234] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-06-06_12-42-51_999086_23201 is over 95% full, available space: 8589963264; capacity: 245107195904. Object creation